In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
from pymongo import MongoClient
from bson.objectid import ObjectId
from datetime import timedelta
from scipy.stats import wilcoxon
from cliffs_delta import cliffs_delta
import numpy as np
import datetime
import re
client = MongoClient()
db = client.smartshark_2_2
vcs_systems = pd.DataFrame(list(db.vcs_system.find()))
travisBuilds = pd.DataFrame(list(db.travis_build.find()))
tags = pd.DataFrame(list(db.tag.find()))
issues = pd.DataFrame(list(db.issue.find()))

In [152]:
# IC
projetosComBuilds = travisBuilds['vcs_system_id'].unique()

# Tags of project with IC
idsVcsComBuilds = vcs_systems.query('_id.isin(@projetosComBuilds)').drop_duplicates().dropna()._id
tagsDosProjComBuild = tags.query('vcs_system_id.isin(@idsVcsComBuilds)').drop_duplicates().dropna()

#Release Tags
tagsDeRelase = tagsDosProjComBuild[
    tagsDosProjComBuild['name']
    .str.contains('rc|RC|candidate|CANDIDATE|beta|BETA|alpha|ALPHA')==False].drop_duplicates().dropna()

# Valid Builds of Release Tags
commitsDeRelease = tagsDeRelase.commit_id.unique()
buildsIntegrados = travisBuilds.query('commit_id.isin(@commitsDeRelease)')
stateIndesejados = ['failed', 'canceled']
BuildsValidos = buildsIntegrados.query('~state.isin(@stateIndesejados)')

#Projects with valid builds for their releases
vcsComEntrega = BuildsValidos.vcs_system_id.unique()

# Aggregate builds with the same commit
BuildsValidosPorCommit = BuildsValidos.groupby(['commit_id'], as_index=False, sort=False).max()

# get the projects with > 2 valid builds for an unique commit (for time to market measurement)
vcsComCD = BuildsValidosPorCommit[['vcs_system_id','_id']].groupby(['vcs_system_id'], as_index=False).agg('count').sort_values(by=['_id'], ascending=False)
vcsComCD = vcsComCD.query('_id > 1')

In [153]:
count = 0
vcsList= []
for i in range(vcsComCD.shape[0]):
    vcs = vcsComCD.iloc[i,0]
    commit_id_DaBuilld = BuildsValidosPorCommit.query('vcs_system_id == @vcs').sort_values(by=['started_at']).iloc[0,0]
    data_de_adocao = tagsDeRelase.query('commit_id == @commit_id_DaBuilld').iloc[0,6]
    a = tagsDeRelase.query('vcs_system_id == @vcs & date >= @data_de_adocao').count().iloc[0]
    b = BuildsValidosPorCommit.query('vcs_system_id == @vcs & started_at >= @data_de_adocao').count().iloc[0]
    
    if (a <= b ):
        vcsList.append(vcs)


In [154]:
#Select the projects with CD
vcsList= []
for index, row in vcsComCD.iterrows():
    vcsList.append(row.vcs_system_id)
    
issue_system = pd.DataFrame(list(db.issue_system.find()))
projects = pd.DataFrame(list(db.project.find()))
listaDosProjetosComCD = [] 
issues_system_ids = []
for vcs in vcsList:
    # Buscar informações sobre o projeto
    project_id = vcs_systems.query('_id == @vcs').iloc[0,2] 

    #nome do projeto
    name = projects.query('_id == @project_id').iloc[0,1]     

    # Informações sobre o issue system
    issues_system_id = issue_system.query('project_id == @project_id').iloc[0,0]
    issues_system_ids.append(issues_system_id)   

    # Pegar o commit da build considerada como a build de adoção (Analisar necessidade de tirar a primeira)
    commit_id_DaBuilld = BuildsValidosPorCommit.query('vcs_system_id == @vcs').sort_values(by=['started_at']).iloc[0,0]
    

    # Pegar a release com o commit de build de adoção
    id_tag_Adocao_CD = tagsDeRelase.query('commit_id == @commit_id_DaBuilld').iloc[0,0]
    nome_da_releasa = tagsDeRelase.query('commit_id == @commit_id_DaBuilld').iloc[0,1]
    data_de_adocao = tagsDeRelase.query('commit_id == @commit_id_DaBuilld').iloc[0,6]

    listaDosProjetosComCD.append([name , vcs,project_id, issues_system_id, commit_id_DaBuilld, id_tag_Adocao_CD, nome_da_releasa, data_de_adocao  ])

dfProjetosComCD = pd.DataFrame(listaDosProjetosComCD, columns=['name', 'vcs_system_id', 'project_id', 'issues_system_id', 'commit_id', 'tag_id','adoption_release', 'cd_adoption_date' ])

In [155]:
issuesProjetosCD = issues.query('issue_system_id.isin(@issues_system_ids)')
media_bugs_no_cd = []
mediana_bugs_no_cd = []
media_bugs_cd = []
mediana_bugs_cd = []
for i in range(dfProjetosComCD.shape[0]):
    tmp_name = dfProjetosComCD.iloc[i,0]    
    tmp_date = dfProjetosComCD.iloc[i,7]
    tmp_issue_sistem = dfProjetosComCD.iloc[i,3]
    df_bugs_cd = issuesProjetosCD.query('issue_system_id == @tmp_issue_sistem & issue_type == \'Bug\' & resolution == \'Fixed\' &  status.isin([\'Closed\',\'Resolved\']) & created_at >= @tmp_date')
    #init_date = tmp_date - (1*(df_bugs_cd.updated_at.max() - tmp_date))
    
    df_bugs_no_cd = issuesProjetosCD.query('issue_system_id == @tmp_issue_sistem & issue_type == \'Bug\' & resolution == \'Fixed\' &  status.isin([\'Closed\',\'Resolved\']) &  updated_at < @tmp_date')
    #df_bugs_no_cd = issuesProjetosCD.query('issue_system_id == @tmp_issue_sistem & issue_type == \'Bug\' & resolution == \'Fixed\' &  status.isin([\'Closed\',\'Resolved\']) & created_at >= @init_date &  updated_at < @tmp_date')
    df_bugs_no_cd['delta'] = df_bugs_no_cd.updated_at - df_bugs_no_cd.created_at
    media_bugs_no_cd.append(df_bugs_no_cd.delta.mean())
    mediana_bugs_no_cd.append(df_bugs_no_cd.delta.median())

    
    df_bugs_cd['delta'] = df_bugs_cd.updated_at - df_bugs_cd.created_at
    media_bugs_cd.append(df_bugs_cd.delta.mean())
    mediana_bugs_cd.append(df_bugs_cd.delta.median())

dfProjetosComCD['mean_delta_bugs_no_cd'] = media_bugs_no_cd
dfProjetosComCD['median_delta_bugs_no_cd'] = mediana_bugs_no_cd
dfProjetosComCD['mean_delta_bugs_cd'] = media_bugs_cd
dfProjetosComCD['median_delta_bugs_cd'] = mediana_bugs_cd

In [156]:
df_bugs_cd.created_at.max(), tmp_date, df_bugs_cd.created_at.max() - tmp_date, tmp_date - (df_bugs_cd.created_at.max() - tmp_date), init_date

(Timestamp('2021-05-24 17:10:36.453000'),
 Timestamp('2017-11-02 15:21:49'),
 Timedelta('1299 days 01:48:47.453000'),
 Timestamp('2014-04-13 13:33:01.547000'),
 Timestamp('2017-10-18 00:00:23.730000'))

In [157]:
print('Projetos tempo medio de correção de bugs maior após CD:', dfProjetosComCD.query('mean_delta_bugs_no_cd < mean_delta_bugs_cd').count().iloc[0] )
print('Projetos tempo medio de correção de bugs menor após CD:', dfProjetosComCD.query('mean_delta_bugs_no_cd >= mean_delta_bugs_cd').count().iloc[0] )
print('Projetos com mediana do tempo de correção de bugs maior após CD:', dfProjetosComCD.query('median_delta_bugs_no_cd < median_delta_bugs_cd').count().iloc[0] )
print('Projetos com mediana do tempo de correção de bugs maior após CD:', dfProjetosComCD.query('median_delta_bugs_no_cd >= median_delta_bugs_cd').count().iloc[0] )

Projetos tempo medio de correção de bugs maior após CD: 6
Projetos tempo medio de correção de bugs menor após CD: 18
Projetos com mediana do tempo de correção de bugs maior após CD: 8
Projetos com mediana do tempo de correção de bugs maior após CD: 16


In [158]:
cliffs_delta((dfProjetosComCD['median_delta_bugs_no_cd'] / np.timedelta64(1,'D')), (dfProjetosComCD['median_delta_bugs_cd'] / np.timedelta64(1,'D')))

(0.4375, 'medium')

In [159]:
cliffs_delta((dfProjetosComCD['mean_delta_bugs_no_cd'] / np.timedelta64(1,'D')), (dfProjetosComCD['mean_delta_bugs_cd'] / np.timedelta64(1,'D')))

(0.59375, 'large')

In [160]:
wilcoxon((dfProjetosComCD['mean_delta_bugs_no_cd'] / np.timedelta64(1,'D')), (dfProjetosComCD['mean_delta_bugs_cd'] / np.timedelta64(1,'D')), alternative='greater')

WilcoxonResult(statistic=269.0, pvalue=0.00013905763626098633)

In [161]:
wilcoxon((dfProjetosComCD['median_delta_bugs_no_cd'] / np.timedelta64(1,'D')), (dfProjetosComCD['median_delta_bugs_cd'] / np.timedelta64(1,'D')), alternative='greater' )

WilcoxonResult(statistic=250.0, pvalue=0.0015715360641479492)

In [162]:
tabela1 = dfProjetosComCD[['name','mean_delta_bugs_no_cd','mean_delta_bugs_cd', 'median_delta_bugs_no_cd','median_delta_bugs_cd' ]].to_csv('tabela1.csv', index=False)

In [163]:
dfProjetosComCD

,name,vcs_system_id,project_id,issues_system_id,commit_id,tag_id,adoption_release,cd_adoption_date,mean_delta_bugs_no_cd,median_delta_bugs_no_cd,mean_delta_bugs_cd,median_delta_bugs_cd
0,nifi,5bd0268235e3ea2b7bbfdbae,5bd022e30da3ef1a10738359,5bdafdb2f16c872096848fba,5bd02869bad86c1decdd6b26,5bd02869bad86c1decdd6b28,nifi-0.7.0,2016-07-12 22:50:16,23 days 10:29:28.479675,4 days 17:18:29.639500,40 days 05:20:48.521923076,7 days 07:29:08.239000
1,kylin,5b43737430a71b06bcbdd168,5b43728b0da3ef07106a790f,5b45f47356677a15366cae54,5b437e9f6d2aba30574abddf,5b437e9f6d2aba30574abde1,kylin-1.5.4.1,2016-09-24 06:59:42,45 days 10:20:59.676813725,19 days 07:44:57.101500,106 days 02:13:49.463194836,96 days 06:37:06.932500
2,struts,5b23df3c30a71b06bc01e16a,5b2225520da3ef3e52ece5e2,5b3514966e373d64706ea578,5b23df61a1501a2544d0d306,5b23df61a1501a2544d0d308,STRUTS_2_5_3,2016-10-05 05:44:50,1205 days 18:22:08.102540704,1274 days 13:46:16.515000,80 days 14:21:49.555500,72 days 02:21:57.183000
3,storm,5b7ffe9530a71b06bc70c038,5b7ffd4e0da3ef049126bdbd,5b868868e68f507fbce49133,5b80162b58a6d702fae4479a,5b80162c58a6d702fae4479c,v0.10.2,2016-09-02 19:13:24,192 days 17:53:44.031920520,81 days 03:52:44.965000,38 days 01:22:49.224124031,6 days 19:40:17.581000
4,opennlp,5b27c64830a71b06bceb0ead,5b27c4cc0da3ef789daec0a1,5b3b1dcd428fc37a4b3c804e,5b27c64ceee553716a77d0e8,5b27c64ceee553716a77d0ea,opennlp-1.7.1,2017-01-20 21:58:00,67 days 10:09:38.633084159,1 days 08:22:53.921500,14 days 00:00:11.716573770,2 days 19:56:33.896000
5,commons-compress,5bed61cc35e3ea2b7b2a1805,5bed583a0da3ef21a58b337c,5bed83e88fc5e9768f66c998,5bed61e465108826d67b10d4,5bed61e465108826d67b10d6,1.13,2016-12-29 16:48:23,195 days 18:10:34.876993974,15 days 14:57:39.539500,32 days 03:28:29.912156250,2 days 23:35:21.944000
6,httpcomponents-core,5d8ccfb33dfe5ddb8590d841,5d8cb4a10da3ef0b68077801,5d8dbf905227b202863a5e83,5d8ccfbb53cc927575cbe9d1,5d8ccfbb53cc927575cbe9d3,4.4.10,2018-06-29 10:25:57,257 days 12:15:10.581994900,187 days 18:14:55.167500,13 days 18:20:48.264718750,2 days 19:17:39.989500
7,manifoldcf,5c3f0a7c621a9a77b35f3add,5c3f09db0da3ef512137483f,5c57ee84cf244d7545c7f950,5c3f2e2566ca241a78524fe1,5c3f2e2566ca241a78524fe3,release-2.7,2017-04-20 21:54:27,21 days 23:48:52.627689935,0 days 08:22:53.303000,24 days 21:25:39.299867647,0 days 14:04:39.134500
8,curator,608b9b33c26a57681ebf90b6,608b9b0227ad48a9ceb48654,60fac2459b37c1166a9cbc13,608b9b38e61739c47b70f055,608b9b38e61739c47b70f057,apache-curator-4.1.1,2019-03-03 19:41:28,102 days 11:28:49.599834285,17 days 14:39:54.749000,141 days 20:16:44.180187500,6 days 04:53:17.152000
9,httpcomponents-client,5cd2806d504acf99a4cd728c,5cd27fe20da3ef671cda02e7,5d7778c99f03373d37512b7e,5cd284c74eb92640bb5323de,5cd284c74eb92640bb5323e2,4.5.7,2019-01-24 09:47:55,1281 days 02:09:34.997129456,1109 days 14:54:45.574000,31 days 10:13:59.393454545,10 days 17:42:49.218000


CSV for Visualization

In [164]:
bugsCDProjects = issuesProjetosCD.query('issue_type == \'Bug\' & status == \'Closed\'')
bugsCDProjects['post_CD'] = False 
for i in range(dfProjetosComCD.shape[0]):
    tmp_date = dfProjetosComCD.loc[i,'cd_adoption_date']
    tmp_issue_sistem =  dfProjetosComCD.loc[i,'issues_system_id']
    bugsCDProjects.loc[(bugsCDProjects['issue_system_id']== tmp_issue_sistem) & (bugsCDProjects['created_at']>= tmp_date),'post_CD'] = True


In [165]:
bugsCDProjects[['_id','external_id','issue_system_id', 'created_at', 'updated_at','priority', 'affects_versions',  'resolution', 'original_time_estimate','post_CD'] ].to_csv('bugs_of_CD_projects.csv', index=False, sep=';')

Count Team contributors per project 

In [166]:
#dfProjetosComCD[['name','cd_adoption_date']].to_csv('contributors.csv',index=False)

In [167]:
issues.created_at.min(), issues.updated_at.max()

(Timestamp('2001-01-22 14:46:21'), Timestamp('2021-10-27 14:25:20.412000'))